In [1]:
import pyptvgtfs
import geopandas as gpd 
import pandas as pd
import os
from shapely.geometry import Point
from tqdm import tqdm

In [2]:
ptv = pyptvgtfs.process_gtfs_zip('http://data.ptv.vic.gov.au/downloads/gtfs.zip', '')
ptv = ptv.set_index(['branch_id', 'table_name'], inplace=False)['df']
# 30s

In [3]:
gdf : gpd.GeoDataFrame = gpd.GeoDataFrame(pd.concat([gpd.read_file(f'../data/core/mb-geojson/{file}') for file in os.listdir('../data/core/mb-geojson')], ignore_index=True))
# 5m

/tmp/ipykernel_101814/3014894134.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gdf : gpd.GeoDataFrame = gpd.GeoDataFrame(pd.concat([gpd.read_file(f'../data/core/mb-geojson/{file}') for file in os.listdir('../data/core/mb-geojson')], ignore_index=True))


In [4]:
branch_ids = ptv.index.get_level_values('branch_id').unique()

In [5]:
for branch_id in branch_ids:
    ptv[branch_id]['stops']['branch_id'] = branch_id

In [6]:
[ptv[branch_id]['stops']['stop_id'].is_unique for branch_id in branch_ids]

[True, True, True, True, True, True, True, True, True, True]

In [7]:
df_stops : pd.DataFrame = pd.concat([ptv[branch_id]['stops'] for branch_id in branch_ids])

/tmp/ipykernel_101814/620580785.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_stops : pd.DataFrame = pd.concat([ptv[branch_id]['stops'] for branch_id in branch_ids])


In [8]:
gdf_vic = gdf[gdf['STE_CODE21'] == '2']

In [18]:
df_stops_points = df_stops.set_index(['stop_id', 'branch_id']).apply(lambda x: Point(x['stop_lon'], x['stop_lat']), axis=1)

In [10]:
df_stmb : pd.DataFrame = df_stops_points.apply(lambda p: gdf_vic[gdf_vic.geometry.contains(p)]['MB_CODE21'].unique())
# 5m - 10m

In [26]:
df_stmb.apply(lambda x: len(x)).value_counts()

1    27952
0      371
Name: count, dtype: int64

In [11]:
df_stmbx : pd.Series = df_stmb.apply(lambda x: x[0] if len(x) == 1 else None)
df_stmbx.rename('mb_id', inplace=True)
df_stmbx : pd.DataFrame = df_stmbx.to_frame().reset_index()

In [15]:
df_stops['mb_id'] = df_stmbx['mb_id']

In [20]:
os.makedirs('../data/ptv/', exist_ok=True)

In [21]:
df_stops.to_csv('../data/ptv/stops_mb.csv', index=False)